# IEMS 450-2 Programming Assignment 1

Created: Feb 14, 2024

Author: Wenhao Gu

This is an implementation of the steepest descent, Newton's ,method with the Hessian modification and L-BFGS algorithm. We use the backtracking line search method.

In [1]:
# Make NumPy module available for numerical computations
import numpy as np

### Initialize options

In [2]:
def minimizeObjInitOptions():
    '''
    Option initialization for minimize_obj function

    Initialize algorithm options with default values

    Return values:
        options:
            This is a dictionary with fields that correspond to algorithmic
            options of our method.  In particular:

            (General Parameters)
            max_iter:
                Maximum number of iterations
            tol:
                Convergence tolerance
            step_type:
                Different ways to calculate the search direction:
               'Steepest Descent'
               'Newton'
               'L-BFGS'
            init_alpha:
                first trial step size to attempt during line search
            suff_decrease_c1:
                coefficient in sufficient decrease condition
            output_level:
                Amount of output printed
                0: No output
                1: Only summary information
                2: One line per iteration (good for debugging)

            (backtracking line search Parameters)
            rho:
                Decrease rate of the line search stepsize
            alpha_tol
                Tolerance to conclude that the stepsize reduces to 0

            (Algorithmic-specific Search Direction Parameters)
            Newton_beta:
                The parameter of the Newton’s method with the Hessian modification
            L-BFGS_init_gamma
                The initial value of parameter gamma for initializing H_k0
            L-BFGS_m
                Memory level for L-BFGS
    '''
    # we first need to create an empty dictionary before we can add entries
    options = {}

    # Now set a default value for each of the options
    options['max_iter'] = 3000
    options['tol'] = 1e-6
    options['step_type'] = 'Steepest Descent'
    options['init_alpha'] = 1.
    options['suff_decrease_c1'] = 1e-4
    options['output_level'] = 2

    # Parameters for backtracking line search
    options['rho'] = 0.5
    options['alpha_tol'] = 1e-10

    # Algorithmic-specific Parameters
    options['Newton_beta'] = 1e-4
    options['L-BFGS_init_gamma'] = 1
    options['L-BFGS_m'] = 6

    return options

### Find Search Direction

In [3]:
def search_direction(step_type, **kwargs):
    num_func_it_search_direction = 0
    if step_type == 'Steepest Descent':
        p_k = -np.copy(kwargs['grad_k'])
        return p_k, num_func_it_search_direction
    elif step_type == 'Newton':
        # get inputs
        options = kwargs['options']
        objFunc = kwargs['objFunc']
        x_k = kwargs['x_k']
        grad_k = kwargs['grad_k']
        num_vars = kwargs['num_vars']
        # start Newton with Hessian Modification
        beta = options['Newton_beta']
        Hess_k = objFunc.hessian(x_k)
        num_func_it_search_direction += 1
        # search tau for modification of Hessian
        if min(np.diagonal(Hess_k)) > 0:
            tau_k = 0
        else:
            tau_k = -min(np.diagonal(Hess_k)) + beta
        while True:
            try:
                L = np.linalg.cholesky(Hess_k + tau_k*np.identity(num_vars))
                break
            except np.linalg.LinAlgError:
                tau_k = max(2*tau_k, beta)
        p_k = np.linalg.solve(L.T, np.linalg.solve(L, -grad_k))
        return p_k, num_func_it_search_direction
    elif step_type == 'L-BFGS':
        # get inputs
        options = kwargs['options']
        iter_count = kwargs['iter_count']
        num_vars = kwargs['num_vars']
        x_k = kwargs['x_k']
        grad_k = kwargs['grad_k']
        s_list = kwargs['s_list']
        y_list = kwargs['y_list']
        # initialize H_k0
        if iter_count == 0:
            gamma_k = options['L-BFGS_init_gamma']
        else:
            s_prev = s_list[-1]
            y_prev = y_list[-1]
            gamma_k = np.dot(s_prev, y_prev) / np.dot(y_prev, y_prev)
        H_k0 = gamma_k*np.identity(num_vars)
        q = np.copy(grad_k)
        alpha_list = np.zeros(len(s_list))
        for i in reversed(range(len(s_list))):
            s_i = s_list[i]
            y_i = y_list[i]
            rho_i = 1 / np.dot(s_i, y_i)
            alpha_list[i] = rho_i*np.dot(s_i, q)
            q = q - alpha_list[i]*y_i
        r = H_k0 @ q
        for i in range(len(s_list)):
            s_i = s_list[i]
            y_i = y_list[i]
            rho_i = 1 / np.dot(s_i, y_i)
            beta = rho_i * np.dot(y_i, r)
            r = r + (alpha_list[i]-beta)*s_i
        # compute the descent direction
        p_k = -r
    
    return p_k, num_func_it_search_direction


In [4]:
def myFun(arg1, **kwargs):
    for key, value in kwargs.items():
        print("%s == %s" % (key, value))
 
 
# Driver code
myFun("Hi", first='Geeks', mid='for', last='Geeks')

first == Geeks
mid == for
last == Geeks


### Backtracking line search

In [5]:
def backtracking(objFunc, x_k, p_k, options):
    '''
    Implementation of the backtracking line search method.
    
    Input arguments:
        objFunc:
            Objective function object.
        x_k:
            The current x value
        p_k:
            The searching direction
        options:
            This is a dictionary with options for the algorithm. See the "minimizeObjInitOptions" function.
            
    Return value:
        alpha_k: the resulting step length
        num_func_it: the number of function evaluations
    '''

    alpha_k = options['init_alpha']
    c = options['suff_decrease_c1']
    rho = options['rho']
    num_func_it = 0
    Df_k = objFunc.gradient(x_k)
    f_k = objFunc.value(x_k)
    num_func_it += 2
    while objFunc.value(x_k+alpha_k*p_k) > f_k + c*alpha_k*np.dot(Df_k, p_k):
        num_func_it += 1
        alpha_k *= rho
    return alpha_k, num_func_it

### Line Search Algorithm

In [6]:
def minimizeObjective(x_start, objFunc, options):
    '''
    Optimization method for unconstrained optimization

    This is the template for your solution.  You need to add or change
    things, not only at the places pointed out explicitly

    Input arguments:
        x_start:
            Starting point
        objFunc:
            Objective function object.  It must have the following methods,
            where x is the point where the quantity should be evaluated.

            val = value(x)
                returns value of objective function at x
            grad = gradient(x)
                returns gradient of objective function at x
            Hess = hessian(x)
                return Hessian of objective function at x
        options:
            This is a dictionary with options for the algorithm.
            For details see the minimizeObjInitOptions function.

    Return values:
        status:
           Return code indicating reason for termination:
            0:  Optimal solution found (convergence tolerance satisfied)
           -1:  Maximum number of iterations exceeded
           -2:  Search direction is not a descent direction
           -3:  Step size becomes zero
          -99:  Unknown error (useful in debugging)
        x_sol:
            Approximate critical point (or last iterate if there is a failure)
        f_sol:
            objective function value at x_sol
        stats:
            Dictionary with statistics for the run.  Its fields are
            norm_grad      Norm of gradient at final iterate
            num_iter       Number of iterations taken
            num_func_evals Number of function evaluations needed
    '''

    # initialize iteration counter
    iter_count = 0

    # initialize return flag (set to -99 so that we do not accidentally
    # declare success.)
    status = -99
    
    # get parameters out put the options dictionary
    max_iter = options['max_iter']
    tol = options['tol']
    step_type = options['step_type']
    output_level = options['output_level']

    # initialize current iterate
    x_k = np.copy(x_start)

    # initialize current function value, gradient, and infinity norm of the
    # gradient
    f_k = objFunc.value(x_k)
    grad_k = objFunc.gradient(x_k)
    norm_grad_k = np.linalg.norm(grad_k, np.inf)

    # initialize counter for the total number of function evaluations
    total_num_func_evals = 0

    # determine how many variables are in the problem
    num_vars = len(x_start)

    # initialize search direction and its length to zero
    p_k = np.zeros(num_vars)
    norm_pk = 0.0

    # initialize step size to zero (this is only necessary so that we can print
    # an output line before a meaningful step size has actually been computed.)
    alpha_k = 0.0

    # initialize the counter for the function evaluations needed in a
    # particular iteration
    num_func_it = 0

    # Print header and zero-th iteration for output
    if output_level >= 2:
        # (This is just a fancy way to create a string.  The '%s' formatting
        # makes it easy to align the header with the actual output.)
        output_header = '%6s %23s %9s %9s %6s %9s' % \
            ('iter', 'f', '||p_k||', 'alpha', '#func', '||grad_f||')
        print(output_header)
        print('%6i %23.16e %9.2e %9.2e %6i %9.2e' %
              (iter_count, f_k, norm_pk, alpha_k, num_func_it, norm_grad_k))

    ###########################
    # Beginning of main Loop
    ###########################

    # We continue the main loop until the termination tolerance is met.
    while True:

        ##############################################################
        # Check termination tests
        ##############################################################
        if norm_grad_k <= tol:
            # Termination tolerance met
            status = 0
            break

        if iter_count >= max_iter:
            # Set flag to indicate the maximum number of iterations has been
            # exceeded
            status = -1
            # The following command says that we now want to leave the current
            # loop (which is the while loop here).  The program execution will
            # resume immediately after the end of the loop
            break   

        ##############################################################
        # Compute search direction
        ##############################################################
        
        # get the arguments for the search direction function
        if step_type == 'Steepest Descent':
            kwargs = {'grad_k': grad_k}
        elif step_type == 'Newton':
            kwargs = {'options': options, 'objFunc': objFunc, 'x_k': x_k, 'grad_k': grad_k, 'num_vars': num_vars}
        elif step_type == 'L-BFGS':
            # update s_list and y_list
            if iter_count == 0:
                s_list = [] # Initialize. Will store s[max(0,k-m)] to s[max(0,k-1)]
                y_list = [] # Initialize. Will store s[max(0,k-m)] to s[max(0,k-1)]
            else:
                s_prev = x_k - x_k_prev  # get s[k-1]
                y_prev = grad_k - grad_k_prev  # get y[k-1]
                s_list.append(s_prev)
                y_list.append(y_prev)
                m = options['L-BFGS_m']
                if iter_count > m:
                    s_list.pop(0)
                    y_list.pop(0)
            # recording x[k-1], grad[k-1]
            x_k_prev = np.copy(x_k)
            grad_k_prev = np.copy(grad_k)
            kwargs = {
                    'options': options,
                    'iter_count': iter_count,
                    'num_vars': num_vars,
                    'x_k': np.copy(x_k),
                    'grad_k': np.copy(grad_k),
                    's_list': s_list,
                    'y_list': y_list
                    }
        else:
            raise ValueError('Invalid value for options[step_type]')

        # get search direction and number of function evaluations
        p_k, num_func_it_search_direction = search_direction(step_type, **kwargs)
        
        # update number of function evaluations
        num_func_it += num_func_it_search_direction

        ##############################################################
        # Perform the backtracking Armijo line search
        ##############################################################

        # check if p_k is a descent direction, otherwise CANNOT do backtracking
        if np.dot(p_k, grad_k) >= 0:
            status = -2
            break
        
        # compute step size
        alpha_k, num_func_it_line_search = backtracking(objFunc, x_k, p_k, options)
        num_func_it += num_func_it_line_search

        # check if stepsize is zero, otherwise the iteration never ends
        if np.abs(alpha_k) < options['alpha_tol']:
            status = -3
            break

        # Compute trial point and objective value at trial point
        x_trial = x_k + alpha_k*p_k
        f_trial = objFunc.value(x_trial)
        num_func_it += 1

        ##############################################################
        # Update Informations in This Iteration
        ##############################################################

        # Update iterate
        x_k = np.copy(x_trial)
        f_k = f_trial

        # Compute gradient and its norm at the new iterate
        grad_k = objFunc.gradient(x_k)
        norm_grad_k = np.linalg.norm(grad_k, np.inf)
        num_func_it += 1

        # For the output, compute the norm of the step
        norm_pk = np.linalg.norm(p_k, np.inf)

        # Update counter for total number of function evaluations
        total_num_func_evals += num_func_it

        # Increase the iteration counter
        iter_count += 1

        # Iteration output
        if output_level >= 2:
            # Print the output header every 10 iterations
            if iter_count % 10 == 0:
                print(output_header)
            print('%6i %23.16e %9.2e %9.2e %6i %9.2e' %
                  (iter_count, f_k, norm_pk, alpha_k, num_func_it, norm_grad_k))
            
        # reset the counter for the function evaluations in the next iteration
        num_func_it = 0

    ###########################
    # End of main loop
    ###########################

    ###########################
    # Finalize results
    ###########################

    # Set last iterate as the one that is returned, together with its objective
    # value
    x_sol = x_k
    f_sol = f_k

    # Set the statistics
    stats = {}
    stats['num_iter'] = iter_count
    stats['norm_grad'] = norm_grad_k
    stats['num_func_evals'] = total_num_func_evals

    # Final output message
    if output_level >= 1:
        print('')
        print('Final objective.................: %g' % f_sol)
        print('||grad|| at final point.........: %g' % norm_grad_k)
        print('Number of iterations............: %d' % iter_count)
        print('Number of function evaluations..: %d' % total_num_func_evals)
        print('')
        if status == 0:
            print('Exit: Critical point found.')
        elif status == -1:
            print('Exit: Maximum number of iterations (%d) exceeded.' %
                  iter_count)
        elif status == -2:
            print('Exit: Search direction is not a descent direction.')
        elif status == -3:
            print('Exit: Step size becomes zero.')
        else:
            print('ERROR: Unknown status value: %d\n' % status)

    # Return output arguments
    return status, x_sol, f_sol, stats


### Numerical Tests

#### Parameters for Different Tests

In [7]:
# The 1st algorithm: Steepest Descent
options_SD = minimizeObjInitOptions()
options_SD['step_type'] = 'Steepest Descent'

# The 2nd algorithm: Newton with Hessian Modification
options_Newton = minimizeObjInitOptions()
options_Newton['step_type'] = 'Newton'

# The 3rd algorithm: L-BFGS with memory size m=6
options_L_BFGS_6 = minimizeObjInitOptions()
options_L_BFGS_6['step_type'] = 'L-BFGS'
options_L_BFGS_6['L-BFGS_m'] = 6

# The 4th algorithm: L-BFGS with memory size m=20
options_L_BFGS_20 = minimizeObjInitOptions()
options_L_BFGS_20['step_type'] = 'L-BFGS'
options_L_BFGS_20['L-BFGS_m'] = 20

#### Test functions

In [8]:
# The function for the 1st and 2nd test
class Rosenbrock:
    '''
    Implementation of the Rosenbrock objective function

    a*(x[1] - x[0]**2)**2 + (b - x[0])**2

    Parameters:
        a: scalar, see formula
        b: scalar, see formula
    '''

    def __init__(self, a=100.0, b=1.0):
        '''
        Initialize object with parameters a and b in the formula.
        '''
        self._a = a
        self._b = b

    def value(self, x):
        '''
        Compute value of objective function at point x
        '''
        a = self._a
        b = self._b
        val = a*(x[1] - x[0]**2)**2 + (b - x[0])**2
        return val

    def gradient(self, x):
        '''
        Compute gradient of objective function at point x
        '''
        a = self._a
        b = self._b
        D0 = -2*b + 2*b*x[0] - 4*a*x[0]*x[1] + 4*a*x[0]**3
        D1 = 2*a*(x[1] - x[0]**2)
        grad = np.array([D0, D1])
        return grad

    def hessian(self, x):
        '''
        Compute Hessian of objective function at point x
        '''
        a = self._a
        b = self._b
        H00 = 2*b - 4*a*x[1] + 12*a*x[0]**2
        H01 = -4*a*x[0]
        H11 = 2*a
        Hess = np.array([[H00, H01], [H01, H11]])
        return Hess
    
# The function for the 3rd and 4th test
class Func34:
    '''
    Implementation of the function for the 3rd and 4th test

    (x[0]-1)**d + (exp(x[1])-1)/(exp(x[1])+1) + 0.1*exp(-x[1])

    Parameters:
        d: just a exponent in the function.
    '''

    def __init__(self, d):
        '''
        Initialize object with parameters d.
        '''
        self._d = d

    def value(self, x):
        '''
        Compute value of objective function at point x
        '''
        d = self._d
        val = (x[0]-1)**d + (np.exp(x[1])-1)/(np.exp(x[1])+1) + 0.1*np.exp(-x[1])

        return val

    def gradient(self, x):
        '''
        Compute gradient of objective function at point x
        '''
        d = self._d
        D0 = d*(x[0]-1)**(d-1)
        D1 = 2*np.exp(x[1])/(np.exp(x[1])+1)**2 - 0.1*np.exp(-x[1])
        grad = np.array([D0, D1])
        return grad

    def hessian(self, x):
        '''
        Compute Hessian of objective function at point x
        '''
        d = self._d
        H00 = d*(d-1)*(x[0]-1)**(d-2)
        H01 = 0
        H11 = 2*np.exp(x[1])*(1-np.exp(x[1]))/((np.exp(x[1])+1)**3) + 0.1*np.exp(-x[1])
        Hess = np.array([[H00, H01], [H01, H11]])
        return Hess
    

# The function for the 5th to the 8th test
class ExtendedRosenbrock:
    '''
    Implementation of the function for the 3rd and 4th test

    sum([a*(x[i+1] - x[i]**2)**2 + (b - x[i])**2 for i in range(n-1)])

    Parameters:
        n: number of variables.
        a: a parameter in the function.
        b: a parameter in the function.
    '''

    def __init__(self, n=10, a=100.0, b=1.0):
        '''
        Initialize object with parameters n, a, b
        '''
        self._n = n
        self._a = a
        self._b = b

    def value(self, x):
        '''
        Compute value of objective function at point x
        '''
        n = self._n
        a = self._a
        b = self._b
        val = sum([a*(x[i+1] - x[i]**2)**2 + (b - x[i])**2 for i in range(n-1)])
        return val

    def gradient(self, x):
        '''
        Compute gradient of objective function at point x
        '''
        n = self._n
        a = self._a
        b = self._b
        grad = np.zeros(n)
        grad[0] = -2*b + 2*b*x[0] - 4*a*x[0]*x[1] + 4*a*x[0]**3
        grad[n-1] = 2*a*(x[n-1] - x[n-2]**2)
        grad[1:n-1] = np.array([ (-2*b + 2*b*x[i] - 4*a*x[i]*x[i+1] + 4*a*x[i]**3 + 2*a*(x[i] - x[i-1]**2)) for i in range(1, n-1)])
        return grad

    def hessian(self, x):
        '''
        Compute Hessian of objective function at point x
        '''
        n = self._n
        a = self._a
        b = self._b
        Hess = np.zeros((n,n))
        # fill diagonal
        diag = np.zeros(n)
        diag[0] = 2*b - 4*a*x[1] + 12*a*x[0]**2
        diag[n-1] = 2*a
        diag[1:n-1] = np.array([(2*b - 4*a*x[i+1] + 12*a*x[i]**2 + 2*a) for i in range(1,n-1)])
        np.fill_diagonal(Hess, diag)
        # fill Hess[i,i+1] and Hess[i+1,i]
        for i in range(n-1):
            Hess[i,i+1] = -4*a*x[i]
            Hess[i+1,i] = -4*a*x[i]
        return Hess

### Test 1

In [10]:
# initialize objection function and starting points
objFunc1 = Rosenbrock()
x_start = np.array([1.2, 1.2])

##### Steepest Descent

In [11]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc1, options_SD )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  5.7999999999999998e+00  0.00e+00  0.00e+00      0  1.16e+02
     1  4.3097519666189282e-01  1.16e+02  9.77e-04     14  2.81e+01
     2  1.9689420206267062e-02  2.81e+01  9.77e-04     14  3.84e+00
     3  1.2615242780140421e-02  3.84e+00  9.77e-04     14  5.91e-01
     4  1.2412774014682531e-02  5.91e-01  9.77e-04     14  1.39e-01
     5  1.2400369865755413e-02  1.39e-01  1.95e-03     13  1.43e-01
     6  1.2390893483147760e-02  1.43e-01  1.95e-03     13  2.15e-01
     7  1.2386536095077436e-02  2.15e-01  1.95e-03     13  1.97e-01
     8  1.2351382060191813e-02  1.97e-01  9.77e-04     14  7.51e-02
     9  1.2335245359874613e-02  7.51e-02  7.81e-03     11  2.75e-01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  1.2279023224760819e-02  2.75e-01  9.77e-04     14  8.74e-02
    11  1.2262350930776284e-02  8.74e-02  3.91e-03     12  1.65e-01
    12  1.2258791818136789e-02  1.65e-01  1.95

##### Newton with Hessian Modification

In [12]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc1, options_Newton )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  5.7999999999999998e+00  0.00e+00  0.00e+00      0  1.16e+02
     1  3.8384034418534448e-02  2.30e-01  1.00e+00      5  4.00e-01
     2  1.8762343235570654e-02  4.67e-01  5.00e-01      6  4.39e+00
     3  4.2891830020663699e-03  6.47e-02  1.00e+00      5  6.15e-01
     4  9.0327328664285887e-04  1.11e-01  1.00e+00      5  1.14e+00
     5  1.8514093528029176e-05  1.29e-02  1.00e+00      5  3.25e-02
     6  3.3970388403664619e-08  8.40e-03  1.00e+00      5  7.19e-03
     7  3.2266760231438070e-14  8.26e-05  1.00e+00      5  1.36e-06
     8  1.0410321264884865e-25  3.53e-07  1.00e+00      5  1.28e-11

Final objective.................: 1.04103e-25
||grad|| at final point.........: 1.27898e-11
Number of iterations............: 8
Number of function evaluations..: 41

Exit: Critical point found.
[1. 1.]


##### L-BFGS with m=6

In [13]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc1, options_L_BFGS_6 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  5.7999999999999998e+00  0.00e+00  0.00e+00      0  1.16e+02
     1  4.3097519666189282e-01  1.16e+02  9.77e-04     14  2.81e+01
     2  1.6222756807622009e-02  2.21e-02  1.00e+00      4  2.69e+00
     3  1.2444873496129579e-02  2.34e-03  1.00e+00      4  1.14e-01
     4  1.2435043464046600e-02  9.34e-05  1.00e+00      4  8.26e-02
     5  1.2414620500022785e-02  2.11e-04  1.00e+00      4  1.25e-01
     6  1.2355676681161347e-02  8.24e-04  1.00e+00      4  2.22e-01
     7  1.2207535096133188e-02  2.41e-03  1.00e+00      4  4.80e-01
     8  1.1815420496702997e-02  6.93e-03  1.00e+00      4  8.87e-01
     9  1.0769186105447673e-02  1.92e-02  1.00e+00      4  1.48e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  7.4030849711183809e-03  5.95e-02  1.00e+00      4  2.09e+00
    11  3.9740802433692862e-03  4.48e-01  5.00e-01      5  1.90e+00
    12  3.0282244902468695e-03  1.58e-01  1.00

##### L-BFGS with m=20

In [14]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc1, options_L_BFGS_20 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  5.7999999999999998e+00  0.00e+00  0.00e+00      0  1.16e+02
     1  4.3097519666189282e-01  1.16e+02  9.77e-04     14  2.81e+01
     2  1.6222756807622009e-02  2.21e-02  1.00e+00      4  2.69e+00
     3  1.2444873496129579e-02  2.34e-03  1.00e+00      4  1.14e-01
     4  1.2435043464046600e-02  9.34e-05  1.00e+00      4  8.26e-02
     5  1.2414620500022785e-02  2.11e-04  1.00e+00      4  1.25e-01
     6  1.2355676681161347e-02  8.24e-04  1.00e+00      4  2.22e-01
     7  1.2207535096133188e-02  2.41e-03  1.00e+00      4  4.80e-01
     8  1.1815417606288400e-02  6.93e-03  1.00e+00      4  8.87e-01
     9  1.0769163984876021e-02  1.92e-02  1.00e+00      4  1.48e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  7.4042188833689290e-03  5.95e-02  1.00e+00      4  2.10e+00
    11  3.8404885670564881e-03  4.46e-01  5.00e-01      5  1.86e+00
    12  3.0052169382151911e-03  1.56e-01  1.00

### Test 2

In [15]:
# initialize objection function and starting points
objFunc2 = Rosenbrock()
x_start = np.array([-1.2, 1])

##### Steepest Descent

In [16]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc2, options_SD )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  2.4199999999999996e+01  0.00e+00  0.00e+00      0  2.16e+02
     1  5.1011126637109534e+00  2.16e+02  9.77e-04     14  3.83e+01
     2  5.0470111376546116e+00  3.83e+01  1.95e-03     13  4.19e+01
     3  4.1140390714609625e+00  4.19e+01  9.77e-04     14  2.81e+00
     4  4.1080850212978168e+00  2.81e+00  1.95e-03     13  3.33e+00
     5  4.1021527140761478e+00  3.33e+00  1.95e-03     13  2.88e+00
     6  4.0961281680729709e+00  2.88e+00  1.95e-03     13  3.45e+00
     7  4.0901246018753623e+00  3.45e+00  1.95e-03     13  2.93e+00
     8  4.0840108684295400e+00  2.93e+00  1.95e-03     13  3.53e+00
     9  4.0779179741546745e+00  3.53e+00  1.95e-03     13  2.96e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  4.0717040782953378e+00  2.96e+00  1.95e-03     13  3.55e+00
    11  4.0655117295061807e+00  3.55e+00  1.95e-03     13  2.96e+00
    12  4.0591966541883640e+00  2.96e+00  1.95

##### Newton with Hessian Modification

In [17]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc2, options_Newton )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  2.4199999999999996e+01  0.00e+00  0.00e+00      0  2.16e+02
     1  4.7318843252666110e+00  3.81e-01  1.00e+00      5  4.64e+00
     2  4.0873986620721201e+00  4.56e+00  1.25e-01      8  2.60e+01
     3  3.2286725886219698e+00  2.21e-01  1.00e+00      5  1.06e+01
     4  3.2138980914461301e+00  4.82e-01  1.00e+00      5  2.21e+01
     5  1.9425854206216286e+00  6.70e-02  1.00e+00      5  3.49e+00
     6  1.6001936936469916e+00  7.35e-01  2.50e-01      7  7.42e+00
     7  1.1783895610053290e+00  1.44e-01  1.00e+00      5  4.13e+00
     8  9.2241158192295880e-01  2.08e-01  1.00e+00      5  8.63e+00
     9  5.9748861678988285e-01  8.91e-02  1.00e+00      5  1.59e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  4.5262509783027682e-01  2.97e-01  5.00e-01      6  5.21e+00
    11  2.8076243818821206e-01  1.02e-01  1.00e+00      5  1.99e+00
    12  2.1139339642292809e-01  1.77e-01  1.00

##### L-BFGS with m=6

In [18]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc2, options_L_BFGS_6 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  2.4199999999999996e+01  0.00e+00  0.00e+00      0  2.16e+02
     1  5.1011126637109534e+00  2.16e+02  9.77e-04     14  3.83e+01
     2  4.1537884272683572e+00  3.17e-02  1.00e+00      4  6.66e+00
     3  4.1172150366452271e+00  6.68e-03  1.00e+00      4  1.41e+00
     4  4.1140172280775502e+00  1.61e-03  1.00e+00      4  1.41e+00

Final objective.................: 4.11402
||grad|| at final point.........: 1.41456
Number of iterations............: 4
Number of function evaluations..: 26

Exit: Search direction is not a descent direction.
[-1.0270705   1.06194659]


##### L-BFGS with m=20

In [19]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc2, options_L_BFGS_20 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  2.4199999999999996e+01  0.00e+00  0.00e+00      0  2.16e+02
     1  5.1011126637109534e+00  2.16e+02  9.77e-04     14  3.83e+01
     2  4.1537884272683572e+00  3.17e-02  1.00e+00      4  6.66e+00
     3  4.1172150366452271e+00  6.68e-03  1.00e+00      4  1.41e+00
     4  4.1140172280775502e+00  1.61e-03  1.00e+00      4  1.41e+00

Final objective.................: 4.11402
||grad|| at final point.........: 1.41456
Number of iterations............: 4
Number of function evaluations..: 26

Exit: Search direction is not a descent direction.
[-1.0270705   1.06194659]


### Test 3

In [20]:
# initialize objection function and starting points
objFunc3 = Func34(2)
x_start = np.array([-1, 1])

##### Steepest Descent

In [21]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc3, options_SD )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  4.4989051013771535e+00  0.00e+00  0.00e+00      0  4.00e+00
     1  4.3636590062970679e+00  4.00e+00  1.00e+00      4  4.00e+00
     2  4.1999553371145595e+00  4.00e+00  1.00e+00      4  4.00e+00
     3  4.0338143323522493e+00  4.00e+00  1.00e+00      4  4.00e+00
     4  3.9057242974335082e+00  4.00e+00  1.00e+00      4  4.00e+00
     5  3.8348823617033445e+00  4.00e+00  1.00e+00      4  4.00e+00
     6  3.8063795862342880e+00  4.00e+00  1.00e+00      4  4.00e+00
     7  3.7974770057474982e+00  4.00e+00  1.00e+00      4  4.00e+00
     8  3.7951389854960529e+00  4.00e+00  1.00e+00      4  4.00e+00
     9 -2.0518702570358294e-01  4.00e+00  5.00e-01      5  2.02e-02
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10 -2.0548804251187031e-01  2.02e-02  1.00e+00      4  9.52e-03
    11 -2.0555451567693428e-01  9.52e-03  1.00e+00      4  4.43e-03
    12 -2.0556889475610024e-01  4.43e-03  1.00

##### Newton with Hessian Modification

In [22]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc3, options_Newton )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  4.4989051013771535e+00  0.00e+00  0.00e+00      0  4.00e+00
     1  4.3422520963530111e+00  3.56e+03  9.77e-04     15  4.00e+00
     2 -1.4905531518596604e-01  2.00e+00  1.00e+00      5  2.72e-01
     3 -2.0468364924167781e-01  3.79e-01  1.00e+00      5  3.14e-02
     4 -2.0557250127084442e-01  5.60e-02  1.00e+00      5  5.76e-04
     5 -2.0557280900004471e-01  1.07e-03  1.00e+00      5  2.06e-07

Final objective.................: -0.205573
||grad|| at final point.........: 2.06291e-07
Number of iterations............: 5
Number of function evaluations..: 35

Exit: Critical point found.
[ 1.         -1.24477034]


C:\Users\wenha\AppData\Local\Temp\ipykernel_28776\197912126.py:74: RuntimeWarning: overflow encountered in exp
  val = (x[0]-1)**d + (np.exp(x[1])-1)/(np.exp(x[1])+1) + 0.1*np.exp(-x[1])


##### L-BFGS with m=6

In [23]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc3, options_L_BFGS_6 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  4.4989051013771535e+00  0.00e+00  0.00e+00      0  4.00e+00
     1  4.3636590062970679e+00  4.00e+00  1.00e+00      4  4.00e+00
     2  3.5848580046624395e-01  1.98e+00  1.00e+00      4  4.00e-01
     3  2.7511520557543373e-01  2.04e-01  1.00e+00      4  4.13e-01

Final objective.................: 0.275115
||grad|| at final point.........: 0.412676
Number of iterations............: 3
Number of function evaluations..: 12

Exit: Search direction is not a descent direction.
[0.99855147 0.42591584]


##### L-BFGS with m=20

In [24]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc3, options_L_BFGS_20 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  4.4989051013771535e+00  0.00e+00  0.00e+00      0  4.00e+00
     1  4.3636590062970679e+00  4.00e+00  1.00e+00      4  4.00e+00
     2  3.5848580046624395e-01  1.98e+00  1.00e+00      4  4.00e-01
     3  2.7511520557543373e-01  2.04e-01  1.00e+00      4  4.13e-01

Final objective.................: 0.275115
||grad|| at final point.........: 0.412676
Number of iterations............: 3
Number of function evaluations..: 12

Exit: Search direction is not a descent direction.
[0.99855147 0.42591584]


### Test 4

In [25]:
# initialize objection function and starting points
objFunc4 = Func34(4)
x_start = np.array([-1, 1])

##### Steepest Descent

In [26]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc4, options_SD )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  1.6498905101377154e+01  0.00e+00  0.00e+00      0  3.20e+01
     1  1.6482882063045029e+01  3.20e+01  1.25e-01      7  3.20e+01
     2  1.6466288809933442e+01  3.20e+01  1.25e-01      7  3.20e+01
     3  1.6449128607460992e+01  3.20e+01  1.25e-01      7  3.20e+01
     4  1.6431409520249289e+01  3.20e+01  1.25e-01      7  3.20e+01
     5  1.6413145003334797e+01  3.20e+01  1.25e-01      7  3.20e+01
     6  1.6394354468427284e+01  3.20e+01  1.25e-01      7  3.20e+01
     7  1.6375063799186410e+01  3.20e+01  1.25e-01      7  3.20e+01
     8  1.6355305785773144e+01  3.20e+01  1.25e-01      7  3.20e+01
     9  1.6335120446255338e+01  3.20e+01  1.25e-01      7  3.20e+01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  1.6314555201356821e+01  3.20e+01  1.25e-01      7  3.20e+01
    11  1.6293664870084587e+01  3.20e+01  1.25e-01      7  3.20e+01
    12  1.6272511457420400e+01  3.20e+01  1.25

##### Newton with Hessian Modification

In [27]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc4, options_Newton )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  1.6498905101377154e+01  0.00e+00  0.00e+00      0  3.20e+01
     1  1.6328772600956416e+01  3.56e+03  9.77e-04     15  3.20e+01
     2  3.0073376788259232e+00  8.00e-01  1.00e+00      5  9.47e+00
     3  4.1880138661252275e-01  4.44e-01  1.00e+00      5  2.81e+00
     4 -8.2414963324335733e-02  2.96e-01  1.00e+00      5  8.32e-01
     5 -1.8124539409703067e-01  1.97e-01  1.00e+00      5  2.46e-01
     6 -2.0076739371059982e-01  1.32e-01  1.00e+00      5  7.30e-02
     7 -2.0462359116512413e-01  8.78e-02  1.00e+00      5  2.16e-02
     8 -2.0538530918083259e-01  5.85e-02  1.00e+00      5  6.41e-03
     9 -2.0553577199875034e-01  3.90e-02  1.00e+00      5  1.90e-03
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10 -2.0556549304920330e-01  2.60e-02  1.00e+00      5  5.63e-04
    11 -2.0557136387398406e-01  1.73e-02  1.00e+00      5  1.67e-04
    12 -2.0557252354307659e-01  1.16e-02  1.00

C:\Users\wenha\AppData\Local\Temp\ipykernel_28776\197912126.py:74: RuntimeWarning: overflow encountered in exp
  val = (x[0]-1)**d + (np.exp(x[1])-1)/(np.exp(x[1])+1) + 0.1*np.exp(-x[1])


##### L-BFGS with m=6

In [28]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc4, options_L_BFGS_6 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  1.6498905101377154e+01  0.00e+00  0.00e+00      0  3.20e+01
     1  1.6482882063045029e+01  3.20e+01  1.25e-01      7  3.20e+01
     2  4.8280807997519987e-01  2.00e+00  1.00e+00      4  3.63e-01
     3  4.7454434002024565e-01  2.27e-02  1.00e+00      4  3.66e-01

Final objective.................: 0.474544
||grad|| at final point.........: 0.36595
Number of iterations............: 3
Number of function evaluations..: 15

Exit: Search direction is not a descent direction.
[1.00025024 0.93256305]


##### L-BFGS with m=20

In [29]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc4, options_L_BFGS_20 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  1.6498905101377154e+01  0.00e+00  0.00e+00      0  3.20e+01
     1  1.6482882063045029e+01  3.20e+01  1.25e-01      7  3.20e+01
     2  4.8280807997519987e-01  2.00e+00  1.00e+00      4  3.63e-01
     3  4.7454434002024565e-01  2.27e-02  1.00e+00      4  3.66e-01

Final objective.................: 0.474544
||grad|| at final point.........: 0.36595
Number of iterations............: 3
Number of function evaluations..: 15

Exit: Search direction is not a descent direction.
[1.00025024 0.93256305]


### Test 5

In [30]:
# initialize objection function and starting points
n = 10
objFunc5 = ExtendedRosenbrock(n=n)
x_start = -1.0*np.ones(n, dtype=np.float64)

##### Steepest Descent

In [31]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc5, options_SD )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.6360000000000000e+03  0.00e+00  0.00e+00      0  1.20e+03
     1  6.8294211961328983e+02  1.20e+03  1.95e-03     13  1.01e+03
     2  4.0090297165278025e+02  1.01e+03  1.95e-03     13  6.93e+02
     3  3.0242645406936873e+02  6.93e+02  1.95e-03     13  5.50e+02
     4  2.4124111347210166e+02  5.50e+02  1.95e-03     13  4.34e+02
     5  2.1071055558652415e+02  4.34e+02  1.95e-03     13  4.62e+02
     6  1.3467380716748934e+02  4.62e+02  1.95e-03     13  3.31e+02
     7  9.1947806361308224e+01  3.31e+02  1.95e-03     13  2.71e+02
     8  5.7210854420018450e+01  2.71e+02  1.95e-03     13  1.50e+02
     9  2.9327616531852183e+01  1.50e+02  1.95e-03     13  7.04e+01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  2.0210965649304228e+01  7.04e+01  1.95e-03     13  5.02e+01
    11  1.7548579627292714e+01  5.02e+01  3.91e-03     12  5.97e+01
    12  1.3676416447729235e+01  5.97e+01  1.95

##### Newton with Hessian Modification

In [32]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc5, options_Newton )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.6360000000000000e+03  0.00e+00  0.00e+00      0  1.20e+03
     1  6.1112474177674585e+02  1.61e+00  1.00e+00      5  3.54e+02
     2  1.3464835189452128e+02  1.30e+00  1.00e+00      5  1.38e+02
     3  1.6304807302381096e+01  6.89e-01  1.00e+00      5  2.50e+01
     4  9.5188961136391868e+00  9.07e-02  1.00e+00      5  4.05e+00
     5  9.1965894796562111e+00  3.09e-01  5.00e-01      6  5.97e+00
     6  8.7943884234852927e+00  1.90e-01  1.00e+00      5  7.21e+00
     7  8.3272592252890743e+00  1.82e-01  1.00e+00      5  6.47e+00
     8  7.8998520913066059e+00  2.39e-01  1.00e+00      5  9.24e+00
     9  7.1389847761331051e+00  1.56e-01  1.00e+00      5  4.20e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  6.6923881822323557e+00  3.24e-01  5.00e-01      6  5.40e+00
    11  5.9975184069451419e+00  2.00e-01  1.00e+00      5  6.45e+00
    12  5.3100466049832784e+00  1.95e-01  1.00

##### L-BFGS with m=6

In [33]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc5, options_L_BFGS_6 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.6360000000000000e+03  0.00e+00  0.00e+00      0  1.20e+03
     1  6.8294211961328983e+02  1.20e+03  1.95e-03     13  1.01e+03
     2  2.1803616650879184e+02  1.26e+00  1.00e+00      4  5.51e+02
     3  4.1390112539255014e+01  5.41e-01  1.00e+00      4  1.68e+02
     4  1.5789210761236129e+01  1.71e-01  1.00e+00      4  5.66e+01
     5  4.9077836335322553e+00  8.67e-02  1.00e+00      4  3.18e+01
     6  4.3642603806536417e+00  1.07e-01  1.00e+00      4  6.30e+01
     7  9.3820671708778614e-01  5.63e-02  1.00e+00      4  1.06e+01
     8  8.0187565336703703e-01  9.33e-03  1.00e+00      4  4.16e+00
     9  7.0847409879994805e-01  1.18e-02  1.00e+00      4  2.27e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  6.9532526454712729e-01  9.51e-03  1.00e+00      4  5.23e+00
    11  6.7612216914885281e-01  3.69e-03  1.00e+00      4  1.42e+00
    12  6.7322441064235794e-01  1.32e-03  1.00

##### L-BFGS with m=20

In [34]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc5, options_L_BFGS_20 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.6360000000000000e+03  0.00e+00  0.00e+00      0  1.20e+03
     1  6.8294211961328983e+02  1.20e+03  1.95e-03     13  1.01e+03
     2  2.1803616650879184e+02  1.26e+00  1.00e+00      4  5.51e+02
     3  4.1390112539255014e+01  5.41e-01  1.00e+00      4  1.68e+02
     4  1.5789210761236129e+01  1.71e-01  1.00e+00      4  5.66e+01
     5  4.9077836335322553e+00  8.67e-02  1.00e+00      4  3.18e+01
     6  4.3642603806536417e+00  1.07e-01  1.00e+00      4  6.30e+01
     7  9.3820671708778614e-01  5.63e-02  1.00e+00      4  1.06e+01
     8  7.9042960126790396e-01  9.84e-03  1.00e+00      4  4.21e+00
     9  6.9914545089300817e-01  1.28e-02  1.00e+00      4  2.43e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  6.8689901293042399e-01  7.24e-03  1.00e+00      4  3.65e+00
    11  6.7610141035922144e-01  2.10e-03  1.00e+00      4  1.57e+00
    12  6.7024149917752496e-01  3.36e-03  1.00

### Test 6

In [35]:
# initialize objection function and starting points
n = 10
objFunc6 = ExtendedRosenbrock(n=n)
x_start = 2.0*np.ones(n, dtype=np.float64)

##### Steepest Descent

In [36]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc6, options_SD )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.6090000000000000e+03  0.00e+00  0.00e+00      0  1.60e+03
     1  1.1436011525103822e+03  1.60e+03  1.95e-03     13  7.37e+02
     2  9.0056162837741567e+02  7.37e+02  3.91e-03     12  1.31e+03
     3  1.4238164056196348e+02  1.31e+03  1.95e-03     13  1.66e+02
     4  8.2899862491000846e+01  1.66e+02  1.95e-03     13  6.93e+01
     5  7.2628813115213504e+01  6.93e+01  7.81e-03     11  1.75e+02
     6  3.0299359351281211e+01  1.75e+02  1.95e-03     13  7.19e+01
     7  2.0548691398083065e+01  7.19e+01  1.95e-03     13  6.78e+01
     8  1.6930012410891443e+01  6.78e+01  1.95e-03     13  6.22e+01
     9  1.5580912804531071e+01  6.22e+01  1.95e-03     13  7.39e+01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  1.5199538403408440e+01  7.39e+01  1.95e-03     13  7.83e+01
    11  9.5364931860786051e+00  7.83e+01  9.77e-04     14  1.06e+01
    12  9.0566987674946429e+00  1.06e+01  3.91

##### Newton with Hessian Modification

In [37]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc6, options_Newton )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.6090000000000000e+03  0.00e+00  0.00e+00      0  1.60e+03
     1  4.9371259120014219e+02  2.48e+00  1.00e+00      5  4.45e+02
     2  7.6349535471019522e+01  2.82e+00  1.00e+00      5  4.85e+02
     3  7.0445566596805946e+00  3.65e-01  1.00e+00      5  2.21e+01
     4  6.0908230186671357e+00  7.90e+00  2.50e-01      7  1.22e+02
     5  2.2154712666240384e+00  1.54e-01  1.00e+00      5  6.05e+00
     6  1.8279548539978256e+00  4.10e+00  2.50e-01      7  4.42e+01
     7  1.2817898388544613e+00  4.55e-01  1.00e+00      5  1.33e+01
     8  1.0138485142445552e+00  1.22e+00  5.00e-01      6  2.38e+01
     9  6.9139521066920451e-01  5.01e-01  1.00e+00      5  1.51e+01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  4.6830056525129204e-01  5.51e-01  1.00e+00      5  1.76e+01
    11  2.7547142023085019e-01  2.86e-01  1.00e+00      5  6.67e+00
    12  1.9716876709828030e-01  4.95e-01  1.00

##### L-BFGS with m=6

In [38]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc6, options_L_BFGS_6 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.6090000000000000e+03  0.00e+00  0.00e+00      0  1.60e+03
     1  1.1436011525103822e+03  1.60e+03  1.95e-03     13  7.37e+02
     2  3.2626091727016711e+02  9.89e-01  1.00e+00      4  3.13e+02
     3  2.5322449070811660e+02  7.52e-01  1.00e+00      4  6.36e+02
     4  1.4805385430783704e+02  4.56e-01  1.00e+00      4  1.34e+02
     5  1.3036445324395910e+02  1.38e-01  1.00e+00      4  9.44e+01
     6  1.0279598232624903e+02  3.67e-01  1.00e+00      4  1.25e+02
     7  7.1617741872374069e+01  5.49e-01  1.00e+00      4  1.76e+02
     8  4.5038691555891369e+01  7.11e-01  1.00e+00      4  1.41e+02
     9  2.5297474226128067e+01  1.17e-01  1.00e+00      4  9.27e+01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  1.3431785899377335e+01  1.93e-01  1.00e+00      4  3.37e+01
    11  9.8819124948718411e+00  7.47e-02  1.00e+00      4  1.71e+01
    12  9.1613707797231605e+00  8.20e-02  1.00

##### L-BFGS with m=20

In [39]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc6, options_L_BFGS_20 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.6090000000000000e+03  0.00e+00  0.00e+00      0  1.60e+03
     1  1.1436011525103822e+03  1.60e+03  1.95e-03     13  7.37e+02
     2  3.2626091727016711e+02  9.89e-01  1.00e+00      4  3.13e+02
     3  2.5322449070811660e+02  7.52e-01  1.00e+00      4  6.36e+02
     4  1.4805385430783704e+02  4.56e-01  1.00e+00      4  1.34e+02
     5  1.3036445324395910e+02  1.38e-01  1.00e+00      4  9.44e+01
     6  1.0279598232624903e+02  3.67e-01  1.00e+00      4  1.25e+02
     7  7.1617741872374069e+01  5.49e-01  1.00e+00      4  1.76e+02
     8  4.5038700999820762e+01  7.11e-01  1.00e+00      4  1.41e+02
     9  2.5144895116961361e+01  1.17e-01  1.00e+00      4  9.25e+01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  1.3054971901178519e+01  1.99e-01  1.00e+00      4  3.19e+01
    11  1.0291395352608589e+01  6.88e-02  1.00e+00      4  1.91e+01
    12  9.2240134463726360e+00  9.47e-02  1.00

### Test 7

In [40]:
# initialize objection function and starting points
n = 100
objFunc7 = ExtendedRosenbrock(n=n)
x_start = -1.0*np.ones(n, dtype=np.float64)

##### Steepest Descent

In [41]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc7, options_SD )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.9996000000000000e+04  0.00e+00  0.00e+00      0  1.20e+03
     1  2.7260478150099516e+03  1.20e+03  1.95e-03     13  1.01e+03
     2  4.1218111290912884e+02  1.01e+03  1.95e-03     13  6.93e+02
     3  3.0592054380494903e+02  6.93e+02  1.95e-03     13  5.50e+02
     4  2.4240878560960800e+02  5.50e+02  1.95e-03     13  4.34e+02
     5  6.3001787342226052e+01  4.34e+02  9.77e-04     14  1.72e+02
     6  5.4510130266868231e+01  1.72e+02  1.95e-03     13  1.51e+02
     7  1.8204995247486917e+01  1.51e+02  9.77e-04     14  6.37e+01
     8  1.0747723849419449e+01  6.37e+01  9.77e-04     14  3.55e+01
     9  7.4057895623553796e+00  3.55e+01  9.77e-04     14  2.67e+01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  6.0895072449204246e+00  2.67e+01  1.95e-03     13  4.03e+01
    11  4.2597659849159992e+00  4.03e+01  9.77e-04     14  2.58e+01
    12  3.4011039799370346e+00  2.58e+01  9.77

##### Newton with Hessian Modification

In [42]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc7, options_Newton )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.9996000000000000e+04  0.00e+00  0.00e+00      0  1.20e+03
     1  6.9635229668557349e+03  1.61e+00  1.00e+00      5  3.54e+02
     2  1.1325237108949466e+03  1.30e+00  1.00e+00      5  1.38e+02
     3  1.9226928373084786e+02  6.89e-01  1.00e+00      5  2.49e+01
     4  1.0154489281807751e+02  9.05e-02  1.00e+00      5  4.02e+00
     5  9.8702203492734185e+01  3.09e-01  1.00e+00      5  1.87e+01
     6  9.7711703670090046e+01  9.96e-02  1.00e+00      5  1.98e+00
     7  9.7454815465245090e+01  3.50e+00  6.25e-02      9  1.06e+01
     8  9.6803245199273775e+01  1.46e-01  1.00e+00      5  4.22e+00
     9  9.6386677204497303e+01  3.21e-01  5.00e-01      6  6.91e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  9.5705692993438248e+01  1.83e-01  1.00e+00      5  6.41e+00
    11  9.5031409565059008e+01  1.94e-01  1.00e+00      5  5.58e+00
    12  9.4340002314700598e+01  1.92e-01  1.00

##### L-BFGS with m=6

In [43]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc7, options_L_BFGS_6 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.9996000000000000e+04  0.00e+00  0.00e+00      0  1.20e+03
     1  2.7260478150099516e+03  1.20e+03  1.95e-03     13  1.01e+03
     2  3.2685872026094904e+02  1.53e+00  1.00e+00      4  5.70e+02
     3  9.8054029549672890e+01  8.67e-01  1.00e+00      4  3.25e+02
     4  3.0194028823033058e+01  3.04e-01  1.00e+00      4  1.19e+02
     5  1.3428143597239345e+01  1.10e-01  1.00e+00      4  4.40e+01
     6  4.0717161274626550e+00  9.74e-02  1.00e+00      4  3.00e+01
     7  3.1761109336303091e+00  8.26e-02  1.00e+00      4  3.75e+01
     8  1.5131116132014850e+00  2.74e-02  1.00e+00      4  8.69e+00
     9  1.3442474842669649e+00  1.10e-02  1.00e+00      4  3.49e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  1.2659891460073203e+00  1.07e-02  1.00e+00      4  2.50e+00
    11  1.2458532837175027e+00  5.50e-03  1.00e+00      4  3.31e+00
    12  1.2290767526732751e+00  2.18e-03  1.00

##### L-BFGS with m=20

In [44]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc7, options_L_BFGS_20 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  3.9996000000000000e+04  0.00e+00  0.00e+00      0  1.20e+03
     1  2.7260478150099516e+03  1.20e+03  1.95e-03     13  1.01e+03
     2  3.2685872026094904e+02  1.53e+00  1.00e+00      4  5.70e+02
     3  9.8054029549672890e+01  8.67e-01  1.00e+00      4  3.25e+02
     4  3.0194028823033058e+01  3.04e-01  1.00e+00      4  1.19e+02
     5  1.3428143597239345e+01  1.10e-01  1.00e+00      4  4.40e+01
     6  4.0717161274626550e+00  9.74e-02  1.00e+00      4  3.00e+01
     7  3.1761109336303091e+00  8.26e-02  1.00e+00      4  3.75e+01
     8  1.5121536394611157e+00  2.74e-02  1.00e+00      4  8.69e+00
     9  1.3408032710621154e+00  1.11e-02  1.00e+00      4  3.53e+00
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  1.2616951410044290e+00  1.12e-02  1.00e+00      4  2.49e+00
    11  1.2423767804661918e+00  5.30e-03  1.00e+00      4  3.43e+00
    12  1.2228876878325052e+00  2.61e-03  1.00

### Test 8

In [45]:
# initialize objection function and starting points
n = 10000
objFunc8 = ExtendedRosenbrock(n=n)
x_start = 2.0*np.ones(n, dtype=np.float64)

##### Steepest Descent

In [46]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc8, options_SD )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  4.0095990000000000e+06  0.00e+00  0.00e+00      0  1.60e+03
     1  2.3857973918800708e+05  1.60e+03  1.95e-03     13  7.37e+02
     2  5.2133550108321084e+04  7.37e+02  7.81e-03     11  3.46e+04
     3  2.2732175304335444e+04  3.46e+04  2.44e-04     16  1.94e+04
     4  6.6829348227133614e+03  1.94e+04  2.44e-04     16  2.78e+03
     5  4.9287202406094329e+03  2.78e+03  9.77e-04     14  1.35e+03
     6  2.5181795642615893e+03  1.35e+03  1.95e-03     13  9.39e+02
     7  1.1481929007435097e+03  9.39e+02  1.95e-03     13  4.71e+02
     8  3.4777699165079201e+02  4.71e+02  3.91e-03     12  3.54e+02
     9  2.9742167378119836e+02  3.54e+02  3.91e-03     12  3.63e+02
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  2.5946068219086078e+02  3.63e+02  1.95e-03     13  4.20e+02
    11  1.1042154724507174e+02  4.20e+02  9.77e-04     14  1.51e+02
    12  8.1984946350030086e+01  1.51e+02  9.77

##### Newton with Hessian Modification

In [47]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc8, options_Newton )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  4.0095990000000000e+06  0.00e+00  0.00e+00      0  1.60e+03
     1  6.8752825259458099e+05  2.48e+00  1.00e+00      5  4.45e+02
     2  9.7900539135796746e+04  2.82e+00  1.00e+00      5  4.85e+02
     3  8.8968883852773215e+03  3.65e-01  1.00e+00      5  2.25e+01
     4  6.7657852387780031e+02  7.88e+00  1.00e+00      5  1.05e+03
     5  6.7816518890035105e-01  2.03e+00  1.00e+00      5  6.80e+00
     6  3.3842702609523312e-01  6.33e-01  1.00e+00      5  2.23e+01
     7  1.3972062634200099e-02  3.08e-02  1.00e+00      5  2.68e-01
     8  1.1476535458411976e-02  2.25e-01  1.00e+00      5  3.71e+00
     9  6.0987362971449688e-07  9.89e-03  1.00e+00      5  1.74e-03
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  2.2203403903506808e-11  1.35e-03  1.00e+00      5  1.59e-04
    11  2.1044404071849174e-23  3.62e-07  1.00e+00      5  8.11e-12

Final objective.................: 2.10444e-23

##### L-BFGS with m=6

In [48]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc8, options_L_BFGS_6 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  4.0095990000000000e+06  0.00e+00  0.00e+00      0  1.60e+03
     1  2.3857973918800708e+05  1.60e+03  1.95e-03     13  7.37e+02
     2  1.7245995126209680e+04  1.16e+00  1.00e+00      4  2.86e+02
     3  1.1937076592454141e+04  6.04e-01  1.00e+00      4  4.38e+02
     4  1.0082820305800200e+04  1.90e+00  1.00e+00      4  2.37e+02
     5  1.0004681051878819e+04  1.11e+00  5.00e-01      5  1.73e+02
     6  9.9615650270761489e+03  2.49e-01  1.00e+00      4  7.13e+01
     7  9.9388446581763201e+03  3.13e-01  1.00e+00      4  5.46e+01
     8  9.9195851042667800e+03  8.63e-01  1.00e+00      4  6.06e+01
     9  9.9040495331978473e+03  3.80e-01  1.00e+00      4  4.18e+01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  9.8993653666975151e+03  1.03e-01  1.00e+00      4  2.49e+01
    11  9.8983088331430336e+03  7.64e-02  1.00e+00      4  1.05e+01
    12  9.8979172479358058e+03  2.80e-02  1.00

##### L-BFGS with m=20

In [49]:
status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc8, options_L_BFGS_20 )
print(x_sol)

  iter                       f   ||p_k||     alpha  #func ||grad_f||
     0  4.0095990000000000e+06  0.00e+00  0.00e+00      0  1.60e+03
     1  2.3857973918800708e+05  1.60e+03  1.95e-03     13  7.37e+02
     2  1.7245995126209680e+04  1.16e+00  1.00e+00      4  2.86e+02
     3  1.1937076592454141e+04  6.04e-01  1.00e+00      4  4.38e+02
     4  1.0082820305800200e+04  1.90e+00  1.00e+00      4  2.37e+02
     5  1.0004681051878819e+04  1.11e+00  5.00e-01      5  1.73e+02
     6  9.9615650270761489e+03  2.49e-01  1.00e+00      4  7.13e+01
     7  9.9388446581763201e+03  3.13e-01  1.00e+00      4  5.46e+01
     8  9.9195826322729063e+03  8.63e-01  1.00e+00      4  6.06e+01
     9  9.9040814117784812e+03  3.80e-01  1.00e+00      4  4.18e+01
  iter                       f   ||p_k||     alpha  #func ||grad_f||
    10  9.8995963589962321e+03  9.54e-02  1.00e+00      4  2.63e+01
    11  9.8982711777902132e+03  8.91e-02  1.00e+00      4  1.08e+01
    12  9.8979724387967017e+03  2.30e-02  1.00

## Observations
The steepest descent with backtracking line search do not often converge in a reasonable number of iterations (3000), but it seems that the gradient is decreasing slowly. So I think it just converges very slow. The runtime per iteration is the shortest.

The L-BFGS with backtracking line search do not always converge, and the reason of failure is because the search direction is not always a descent direction. Therefore we need to either modify L-BFGS or the backtracking method to make it perform better. The runtime per iteration is longer than the steepest descent method, because it needs to O(m) times of matrix-vector multiplications.

The Newton's method with Hessian modification and together with backtracking performs the best. It always converges in a few number of iterations (around 10 to 20) on our tested examples. The runtime per iteration is the longest, especially when the problem dimension n is large. This is because it requires solving n by n linear systems and do Cholesky factorizations many times.